<a href="https://colab.research.google.com/github/gautaman8/gesture-recognition/blob/main/Neural_Nets_Project_Starter_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [2]:
!pip install -r requirements.txt

  Using cached absl_py-2.0.0-py3-none-any.whl (130 kB)
  Using cached cachetools-5.3.2-py3-none-any.whl (9.3 kB)
  Using cached certifi-2023.7.22-py3-none-any.whl (158 kB)
  Using cached charset_normalizer-3.3.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (141 kB)
  Using cached flatbuffers-23.5.26-py2.py3-none-any.whl (26 kB)
  Using cached google_auth-2.23.4-py2.py3-none-any.whl (183 kB)
  Using cached google_auth_oauthlib-1.0.0-py2.py3-none-any.whl (18 kB)
  Using cached grpcio-1.59.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (5.3 MB)
  Using cached h5py-3.10.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.8 MB)
  Using cached idna-3.4-py3-none-any.whl (61 kB)
  Using cached imageio-2.32.0-py3-none-any.whl (313 kB)
  Using cached importlib_metadata-6.8.0-py3-none-any.whl (22 kB)
  Using cached keras-2.13.1-py3-none-any.whl (1.7 MB)
  Using cached lazy_loader-0.3-py3-none-any.whl (9.1 kB)
  Using cached libclang-16.0.6-py2.py3-none-manylinux2

In [4]:
import numpy as np
import os
# from scipy.misc import  imresize
from imageio import imread
# imresize in starter code needs to be replaced with the below method in skimage library
from skimage.transform import resize

We set the random seed so that the results don't vary drastically.

In [5]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.random.set_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

# Experiment #1

In [6]:
train_doc = np.random.permutation(open('notebooks/storage/Final_data/Collated_training/train.csv').readlines())
val_doc = np.random.permutation(open('notebooks/storage/Final_data/Collated_training/val.csv').readlines())
batch_size = 16

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [7]:
from skimage import io, color, exposure
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = range(30)
    while True:
        t = np.random.permutation(folder_list)
        num_batches = 30
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,1)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)


                    #crop the images and resize them. Note that the images are of 2 different shape
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    height, width, channels = image.shape

                    if height == 360: 
                        new_width = int((160/height)* width)

                        image = resize(image, (160, new_width))
                        crop_left = (new_width - 120)/2
                        crop_right = (new_width - crop_left)
                        image = image[:, crop_left, crop_right]

                    
                    if channels == 3: 
                        image = color.rgb2gray(image)
                    
                    # Additional histogram based preprocessing 
                    preprocessed_image = exposure.equalize_hist(image)
                    # converting color(rgb) to gray to simplify data processing
                    # batch_data[folder,idx,:,:,0] = #normalise and feed in the image
                    
                    normalized_image = (preprocessed_image - preprocessed_image.min()) / (preprocessed_image.max() - preprocessed_image.min())
                    batch_data[folder,idx,:,:,1] = normalized_image
                    # batch_data[folder,idx,:,:,2] = #normalise and feed in the image

                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do


        # write the code for the remaining data points which are left after full batches
        remaining_data_points = len(folder_list) % batch_size
        if remaining_data_points > 0: 
            batch_data_remaining = np.zeros((remaining_data_points, x, y, z, 1))
            batch_labels_remaining = np.zeros((remaining_data_points, 5))

            for folder in range(remaining_data_points):
                imgs = os.listdir(source_path + '/' + t[folder + (num_batches * batch_size)].split(';')[0])

                for idx, item in enumerate(img_idx):
                    image = io.imread(source_path + '/' + t[folder + (num_batches * batch_size)].strip().split(';')[0] + '/' + imgs[item])
                    #crop the images and resize them. Note that the images are of 2 different shape
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    height, width, channels = image.shape

                    if height == 360: 
                        new_width = int((160/height)* width)

                        image = resize(image, (160, new_width))
                        crop_left = (new_width - 120)/2
                        crop_right = (new_width - crop_left)
                        image = image[:, crop_left, crop_right]

                    
                    if channels == 3: 
                        image = color.rgb2gray(image)
                    
                    # Additional histogram based preprocessing 
                    preprocessed_image = exposure.equalize_hist(image)
                    # converting color(rgb) to gray to simplify data processing
                    # batch_data[folder,idx,:,:,0] = #normalise and feed in the image
                    
                    normalized_image = (preprocessed_image - preprocessed_image.min()) / (preprocessed_image.max() - preprocessed_image.min())
                    batch_data_remaining[folder, idx, :, :, 0] = normalized_image

                batch_labels_remaining[folder, int(t[folder + (num_batches * batch_size)].strip().split(';')[2])] = 1
            yield batch_data_remaining, batch_labels_remaining

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [9]:
import datetime
curr_dt_time = datetime.datetime.now()
train_path = 'notebooks/storage/Final_data/Collated_training/train'
val_path = 'notebooks/storage/Final_data/Collated_training/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 50
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 50


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [10]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

model = Sequential()

# Input
model.add(Conv3D(32), kernel_size=(3,3,3), input_shape = (30, 120, 160, 1), padding = 'same')
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size = (2, 2, 2)))

# Convolution
model.add(Conv3D(64, kernel_size=(3,3,3), padding ='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2,2,2)))

# Flatten layer
model.add(Flatten())

# Fully connected layers
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(5, activation = 'softmax'))




2023-11-16 04:10:55.039723: W tensorflow/stream_executor/platform/default/dso_loader.cc:65] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/extras/CUPTI/lib64:/usr/local/cuda/compat/lib:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-11-16 04:10:55.040283: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-11-16 04:10:55.040323: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:163] no NVIDIA GPU device is present: /dev/nvidia0 does not exist


TypeError: __init__() missing 1 required positional argument: 'kernel_size'

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [ ]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [ ]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'

if not os.path.exists(model_name):
    os.mkdir(model_name)

filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [ ]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [ ]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
                    callbacks=callbacks_list, validation_data=val_generator,
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)